In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)
import numpy as np
from force import NeuralNetwork
from force import Simulation
from tqdm import tnrange, tqdm_notebook
from plotly.offline import iplot
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

In [9]:
boc = BrainObservatoryCache(manifest_file='/Users/elijahc/dev/allen-assistant/boc/manifest.json')
exps = boc.get_ophys_experiments(imaging_depths=[175], targeted_structures=['VISp'], cre_lines=['Cux2-CreERT2'],stimuli=['natural_scenes'])

exp_id = exps[1]['id']
print('Experiment ID: %d' % exp_id)
data_set = boc.get_ophys_experiment_data(exp_id)
cids = data_set.get_cell_specimen_ids()
idxs = data_set.get_spontaneous_activity_stimulus_table()
t, dff = data_set.get_dff_traces(cell_specimen_ids=[cids[0]])
print(idxs)
print(dff.shape)

Experiment ID: 500964514


   start    end
0  30752  39654
(1, 114097)


In [10]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def train_force(cell_ids,data_set,net,idx_table,num_train=1000,num_test=200,win=0):
    max_len = num_train+num_test
    t,dff = data_set.get_dff_traces(cell_specimen_ids=cell_ids)
    dff = dff[:,idx_table['start'].item():idx_table['end'].item()].astype(np.float32)
    t = t[idx_table['start'].item():idx_table['end'].item()].astype(np.float32)
    dt = np.around((np.roll(t,-1)-t)[:-1].mean(),decimals=5)
    sim = Simulation(net,dt=dt)
    sim.ft = dff[:,:max_len].T
    print(sim.ft.shape)
    sim.timeline=t[:max_len]
    sim.run('training',t_begin=0,t_end=num_train)
    zt1 = sim.zt
    sim.run('testing',train=False,t_begin=num_train,t_end=max_len)
    zt2 = sim.zt
    sim.zt = np.concatenate([zt1,zt2])
    
    return sim
    
net = NeuralNetwork(N=1000,num_fits=100,pg=0.2)
n_train=7000
n_test=1000
sim = train_force(cell_ids=cids[:100],
                  data_set=data_set,
                  net=net,
                  idx_table=idxs,
                  num_train=n_train,
                  num_test=n_test,win=4)



(8000, 100)


In [4]:
ts1 = sim.timeline
ft_full =[]
zt_full =[]
ft_test_data = []
zt_test_data = []
print(sim.ft.shape)
print(sim.zt.shape)
for i in np.arange(sim.ft.shape[1]):
    fti = Scatter(
        x=ts1,
        y=sim.ft[:,i],
        mode='lines',
        name='ft%d'%i)
    zti = Scatter(
        x=ts1,
        y=sim.zt[:,i],
        mode='lines',
        name='zt%d'%i)
    ft_full.append(fti)
    zt_full.append(zti)
    
    ft_testi = Scatter(
        x=sim.timeline[n_train:],
        y=sim.ft[n_train:,i],
        mode='lines',
        name='ft_test_%d'%i
    )
    zt_testi = Scatter(
        x=sim.timeline[n_train:],
        y=sim.zt[n_train:,i],
        mode='lines',
        name='zt_test_%d'%i
    )
    ft_test_data.append(ft_testi)
    zt_test_data.append(zt_testi)
error = []
for f,z in zip(ft_test_data,zt_test_data):
    diff = f.y - z.y
    sse = np.power(diff,2).sum()
    error.append(sse)
print(np.array(error).shape)

(8000, 100)
(8000, 100)
(100,)


In [11]:
iplot([{'y':np.array(error)}])

#iplot([ft2,zt2])

In [13]:
idx = 37
iplot([ft_test_data[idx],zt_test_data[idx]])
iplot([ft_full[idx],zt_full[idx]])

In [7]:
idx = 57
